In [54]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plot
import ipywidgets as widgets

from ipywidgets import interact
from enum import IntEnum, auto

In [1]:
CRIT=1
FAIL=20

In [99]:
class AttributeResult(IntEnum):
    CRIT_CONFIRMED = auto()
    CRIT_UNCONFIRMED = auto()
    SUCCESS = auto()
    MISS = auto()
    FAIL_UNCONFIRMED = auto()
    FAIL_CONFIRMED = auto()

In [98]:
def attribute_classify(d1: int, d2: int, eew: int) -> AttributeResult:
    if d1 <= CRIT:
        if d2 <= eew or d2 <= CRIT:
            return AttributeResult.CRIT_CONFIRMED
        else:
            return AttributeResult.CRIT_UNCONFIRMED
    elif d1 >= FAIL:
        if d2 > eew or d2 >= FAIL:
            return AttributeResult.FAIL_CONFIRMED
        else:
            return AttributeResult.FAIL_UNCONFIRMED
    elif d1 > eew:
        return AttributeResult.MISS
    else:
        return AttributeResult.SUCCESS
    
def attribute_test(attribute, modificator):
    eew = attribute + modificator
    rolls = [attribute_classify(d1 + 1, d2 + 1, eew) for d1 in range(20) for d2 in range(20)]
    (hist, _) = np.histogram(rolls, bins=len(AttributeResult))
    return [x / sum(hist) for x in hist]

def attribute_plot(attribute, modificator):
    hist = attribute_test(attribute, modificator)
    names = [AttributeResult(i).name for i in range(1, len(AttributeResult) + 1)]
    plot.pie(hist, labels=names, autopct="%0.2f", pctdistance=0.85, radius=2)
    return ["{0:6.2f}: {1}%".format(round(100 * x, 2), names[i]) for (i, x) in enumerate(hist)]
    
interact(attribute_plot,
         attribute=widgets.IntSlider(min=0, max=40, step=1, value=13),
         modificator=widgets.IntSlider(min=-20, max=20, step=1, value=0)
        );

interactive(children=(IntSlider(value=13, description='attribute', max=40), IntSlider(value=0, description='mo…

In [100]:
class TalentResult(IntEnum):
    CRIT_TRIPLE = auto()
    CRIT_DOUBLE = auto()
    QS6 = auto()
    QS5 = auto()
    QS4 = auto()
    QS3 = auto()
    QS2 = auto()
    QS1 = auto()
    MISS = auto()
    FAIL_DOUBLE = auto()
    FAIL_TRIPLE = auto()

In [180]:
def talent_classify(dies: [int], eews: [int], fw: int, talented: bool) -> TalentResult:
    if all([d >= FAIL for d in dies]):
        return TalentResult.FAIL_TRIPLE
    if sum([d >= FAIL for d in dies]) == 2:
        return TalentResult.FAIL_DOUBLE
    if talented:
        dies.remove(max(dies))
    if all([d <= CRIT for d in dies]):
        return TalentResult.CRIT_TRIPLE
    if sum([d <= CRIT for d in dies]) == 2:
        return TalentResult.CRIT_DOUBLE
    fp = fw - sum([max(d - eew, 0) for (d, eew) in zip(dies, eews)])
    if fp < 0:
        return TalentResult.MISS
    if fp < 4:
        return TalentResult.QS1
    if fp < 7:
        return TalentResult.QS2
    if fp < 10:
        return TalentResult.QS3
    if fp < 13:
        return TalentResult.QS4
    if fp < 16:
        return TalentResult.QS5
    return TalentResult.QS6
    
def talent_test(attributes: [int], fw: int, modificator: int, talented):
    eews = [a + modificator for a in attributes]
    if any([eew < 1 for eew in eews]):
        return [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]  # we always miss an impossible feat
    classify = lambda dies: talent_classify(dies, eews, fw, talented)
    rolls = [classify([d + 1 for d in [d1, d2, d3, d4]]) for d1 in range(20) for d2 in range(20) for d3 in range(20) for d4 in range(20)]
    (hist, _) = np.histogram(rolls, bins=len(TalentResult))
    return [x / sum(hist) for x in hist]

def talent_plot(attribute1: int, attribute2: int, attribute3: int, fw: int, modificator: int, talented: bool):
    hist = talent_test([attribute1, attribute2, attribute3], fw, modificator, talented)
    names = [TalentResult(i).name for i in range(1, len(TalentResult) + 1)]
    plot.pie(hist, labels=names, autopct="%0.2f", pctdistance=0.85, radius=2)
    return ["{0:6.2f}%: {1}".format(round(100 * x, 2), names[i]) for (i, x) in enumerate(hist)]
    
interact(talent_plot,
         attribute1=widgets.IntSlider(min=0, max=40, step=1, value=13),
         attribute2=widgets.IntSlider(min=0, max=40, step=1, value=13),
         attribute3=widgets.IntSlider(min=0, max=40, step=1, value=13),
         fw=widgets.IntSlider(min=0, max=40, step=1, value=10),
         modificator=widgets.IntSlider(min=-20, max=20, step=1, value=0),
         talented=widgets.Checkbox(value=False)
        );

interactive(children=(IntSlider(value=13, description='attribute1', max=40), IntSlider(value=13, description='…

In [184]:
def masstalent_classify(dies: [int], eews: [int], fw: int, talented: bool, mastertalented: bool) -> TalentResult:
    fails = sum([d >= FAIL for d in dies[:3]])
    if fails >= 2:
        if mastertalented and fails < 3:
            return 0
        return -1
    if talented:
        dies.remove(max(dies))
    crit = sum([d <= CRIT for d in dies]) >= 2
    fp = fw - sum([max(d - eew, 0) for (d, eew) in zip(dies, eews)])
    if fp < 0:
        return 0
    qs = max(1, fp) // 3
    if crit:
        qs = 2 * qs
    return qs
    
def masstalent_singletest(attributes: [int], fw: int, modificator: int, talented: bool, mastertalented: bool):
    eews = [a + modificator for a in attributes]
    if any([eew < 1 for eew in eews]):
        {0: 1}
    classify = lambda dies: masstalent_classify(dies, eews, fw, talented, mastertalented)
    rolls = [classify([d + 1 for d in [d1, d2, d3, d4]]) for d1 in range(20) for d2 in range(20) for d3 in range(20) for d4 in range(20)]
    hist = {}
    for v in rolls:
        if not v in hist:
            hist[v] = 1
        else:
            hist[v] = hist[v] + 1
    factor = 1.0 / sum(hist.values())
    for k in hist:
      hist[k] = factor * hist[k]
    return hist

def masstalent_test(maxattempts: int, attributes: [int], fw: int, modificator: int, talented: bool, mastertalented: bool, useschip: bool) -> [float]:
    chances = [masstalent_singletest(attributes, fw, modificator - i, talented, mastertalented) for i in range(maxattempts)]
    
    # our result contains the probability mass that we're done after the given amount of time (or never)
    result = {}
    for i in range(maxattempts + 1):
        result[i] = 0.0
    
    # to keep track of all possible states we keep a matrix with the reached amount of qs and the current modificator
    running = np.zeros((10, maxattempts + 1))
    running[(0, 0)] = 1.0
    
    # in each round we redistribute mass from our running state according to the probabilities for the according modificator
    for attempt in range(maxattempts):
        newrunning = np.zeros_like(running)
        for (qs, mod) in np.ndindex(10, attempt + 1): # our round number is also the worst modificator we may have at the moment
            p = running[(qs, mod)]
            for r, chance in chances[mod].items():
                newmod = mod
                newqs = qs + r
                newp = p * chance
                if r <= 0:
                    newmod = newmod + 1
                if r < 0:
                    newqs = 0
                reroll = r == 0 and (mastertalented or useschip)
                if not reroll:
                    if newqs >= 10:
                        result[attempt + 1] = result[attempt + 1] + newp
                    else:
                        newrunning[(newqs, newmod)] = newrunning[(newqs, newmod)] + newp
                else:
                    for mr, mchance in chances[mod].items():
                        newmod = mod
                        newqs = qs + mr
                        mnewp = newp * mchance
                        if mr <= 0:
                            newmod = newmod + 1
                        if mr < 0:
                            newqs = 0
                        mreroll = mr == 0 and mastertalented and useschip
                        if not mreroll:
                            if newqs >= 10:
                                result[attempt + 1] = result[attempt + 1] + mnewp
                            else:
                                newrunning[(newqs, newmod)] = newrunning[(newqs, newmod)] + mnewp
                        else:
                            for sr, schance in chances[mod].items():
                                newmod = mod
                                newqs = qs + sr
                                snewp = mnewp * schance
                                if sr <= 0:
                                    newmod = newmod + 1
                                if sr < 0:
                                    newqs = 0
                                if newqs >= 10:
                                    result[attempt + 1] = result[attempt + 1] + snewp
                                else:
                                    newrunning[(newqs, newmod)] = newrunning[(newqs, newmod)] + snewp
                            continue
                    continue
        running = newrunning
    result[0] = np.sum(running)
    return result.values()

def masstalent_plot(maxattempts: int, attribute1: int, attribute2: int, attribute3: int, fw: int, modificator: int, talented: bool, mastertalented: bool, useschip: bool):
    hist = masstalent_test(maxattempts, [attribute1, attribute2, attribute3], fw, modificator, talented, mastertalented, useschip)
    names = ["miss"] + [str(i + 1) for i in range(maxattempts)]
    plot.pie(hist, labels=names, autopct="%0.2f", pctdistance=0.85, radius=2)
    return ["{0:6.2f}%: {1}".format(round(100 * x, 2), names[i]) for (i, x) in enumerate(hist)]
    
interact(masstalent_plot,
         maxattempts=widgets.IntSlider(min=0, max=10, step=1, value=7),
         attribute1=widgets.IntSlider(min=0, max=20, step=1, value=17),
         attribute2=widgets.IntSlider(min=0, max=20, step=1, value=17),
         attribute3=widgets.IntSlider(min=0, max=20, step=1, value=17),
         fw=widgets.IntSlider(min=0, max=40, step=1, value=19),
         modificator=widgets.IntSlider(min=-20, max=20, step=1, value=0),
         talented=widgets.Checkbox(value=False),
         mastertalented=widgets.Checkbox(value=False),
         useschip=widgets.Checkbox(value=False)
        );

interactive(children=(IntSlider(value=7, description='maxattempts', max=10), IntSlider(value=17, description='…